In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso
from sklearn.metrics import explained_variance_score
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from statsmodels.graphics.regressionplots import influence_plot
from statsmodels.graphics.gofplots import qqplot

/Users/preethi.john/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


	- instant: record index
	- dteday : date
	- season : season (1:springer, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2011, 1:2012)
	- mnth : month ( 1 to 12)
	- hr : hour (0 to 23)
	- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
	- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
	- hum: Normalized humidity. The values are divided to 100 (max)
	- windspeed: Normalized wind speed. The values are divided to 67 (max)
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered


In [37]:
df = pd.read_csv('../Data/hour.csv')

df = df.drop('instant', axis=1)

df.dteday = pd.to_datetime(df.dteday)

In [8]:
df.dteday = pd.to_datetime(df.dteday)


for season in df.season.unique():
    df['season_'+str(season)] = (df['season']==season)
    
df = df.drop('season', axis=1)


for mnth in df.mnth.unique():
    df['mnth_'+str(mnth)] = (df['mnth']==mnth)
    
df = df.drop('mnth', axis=1)


for weekday in df.weekday.unique():
    df['weekday_'+str(weekday)] = (df['weekday']== weekday)
    
df = df.drop('weekday', axis = 1)


for weathersit in df.weathersit.unique():
    df['weathersit_'+str(weathersit)] = (df['weathersit']== weathersit)
    
df = df.drop('weathersit', axis=1)


for hr in df.hr.unique():
    df['hr_'+str(hr)] = (df['hr'] == hr)
    
df = df.drop('hr', axis = 1)

df['outcome'] = df['cnt']
df['casual_users'] = df['casual']
df['registered_users'] = df['registered']
df = df.drop(['cnt', 'casual', 'registered'], axis = 1)

In [36]:
m = ols('outcome ~ temp + windspeed + hum + yr + weekday_0 + weekday_1 + weekday_2 + weekday_3 + weekday_4 + weekday_5 + weekday_6 +  weathersit_1 + weathersit_2 + weathersit_3 +  weathersit_4 + season_1  + season_2  + season_3  + season_4 + mnth_1 + mnth_2 + mnth_3  +  mnth_4 + mnth_5 + mnth_6 + mnth_7 + mnth_8  + mnth_9 + mnth_10 + mnth_11 + mnth_12 + hr_0 + hr_1 + hr_2 + hr_3 + hr_4 + hr_5 + hr_6 + hr_7 + hr_8 + hr_9 + hr_10 + hr_11 + hr_12 + hr_13 + hr_14 + hr_15 + hr_16 + hr_17 + hr_18 + hr_19 + hr_20 + hr_21 + hr_22 + hr_23 - 1',df).fit()

print(m.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     755.3
Date:                Sat, 19 May 2018   Prob (F-statistic):               0.00
Time:                        15:53:11   Log-Likelihood:            -1.0499e+05
No. Observations:               17379   AIC:                         2.101e+05
Df Residuals:                   17328   BIC:                         2.105e+05
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
weekday_0[False]      1.213e+14 

In [23]:
lr = LinearRegression()
grid =GridSearchCV(lr, {}, cv=32 )
grid.fit(df.iloc[:, 1:-3], df['outcome'])
y_pred = grid.predict(df.iloc[:, 1:-3])
explained_variance_score(df.outcome, y_pred)

0.68622361377482033

In [34]:
np.absolute(df.corr().outcome)

yr                  0.250495
holiday             0.030927
workingday          0.030284
temp                0.404772
atemp               0.400929
hum                 0.322911
windspeed           0.093234
season_1            0.245456
season_2            0.060692
season_3            0.151621
season_4            0.029421
mnth_1              0.156834
mnth_2              0.122113
mnth_3              0.057131
mnth_4              0.003645
mnth_5              0.056422
mnth_6              0.084600
mnth_7              0.071459
mnth_8              0.081657
mnth_9              0.084931
mnth_10             0.054406
mnth_11             0.020074
mnth_12             0.079415
weekday_6           0.001692
weekday_0           0.027118
weekday_1           0.012860
weekday_2           0.003969
weekday_3           0.003746
weekday_4           0.015653
weekday_5           0.015034
                      ...   
weathersit_2        0.046902
weathersit_3        0.128034
weathersit_4        0.008340
hr_0          

In [ ]:
m = ols('np.log(outcome) ~ temp + atemp + windspeed + hum + yr + weekday_0 + weekday_1 + weekday_2 + weekday_3 + weekday_4 + weekday_5 + weekday_6 +  weathersit_1 + weathersit_2 + weathersit_3 +  weathersit_4 + season_1  + season_2  + season_3  + season_4 + mnth_1 + mnth_2 + mnth_3  +  mnth_4 + mnth_5 + mnth_6 + mnth_7 + mnth_8  + mnth_9 + mnth_10 + mnth_11 + mnth_12 + hr_0 + hr_1 + hr_2 + hr_3 + hr_4 + hr_5 + hr_6 + hr_7 + hr_8 + hr_9 + hr_10 + hr_11 + hr_12 + hr_13 + hr_14 + hr_15 + hr_16 + hr_17 + hr_18 + hr_19 + hr_20 + hr_21 + hr_22 + hr_23 - 1',df).fit()

print(m.summary())